In [ ]:
import enum
import didppy
import util

In [ ]:
import marimo as mo
import nbformat

# DIDP を用いたモデル

考案者の解説記事: [動的計画法ベースの数理最適化ソルバDIDPPyで最短共通超配列問題を解く](https://zenn.dev/okaduki/articles/7f9a3f3c54bc98)

In [ ]:
class BoundOption(enum.IntFlag):
    MAX_LEN = 0b1
    SCS2LEN = 0b10
    CHAR_COUNT = 0b100

    ALL = 0b111

In [ ]:
def boundexpr_maxlen(
    instance: list[str],
    dpmodel: didppy.Model,
    index_vars: list[didppy.ElementVar]
) -> didppy.IntExpr:
    min_to = dpmodel.add_int_table(
        [
            [
                len(s) - j for j in range(len(s) + 1)
            ]
            for s in instance
        ]
    )

    bound = didppy.IntExpr(0)
    for sidx, index_var in enumerate(index_vars):
        bound = didppy.max(bound, min_to[sidx, index_var])

    return bound

In [ ]:
def boundtable_scs2(s1: str, s2: str) -> list[list[int]]:
    len1, len2 = len(s1), len(s2)

    dp = [[len1 + len2 for _ in range(len2 + 1)] for _ in range(len1 + 1)]

    for i1 in range(len1 + 1):
        dp[i1][len2] = len1 - i1
    for i2 in range(len2 + 1):
        dp[len1][i2] = len2 - i2

    for i1 in range(len1 - 1, -1, -1):
        for i2 in range(len2 - 1, -1, -1):
            if s1[i1] == s2[i2]:
                dp[i1][i2] = dp[i1 + 1][i2 + 1] + 1
            else:
                dp[i1][i2] = min(dp[i1 + 1][i2], dp[i1][i2 + 1]) + 1

    return dp

In [ ]:
def boundexpr_scs2len(
    instance: list[str],
    dpmodel: didppy.Model,
    index_vars: list[didppy.ElementVar]
) -> didppy.IntExpr:
    exprs = []
    for idx1, (s1, index_var1) in enumerate(zip(instance, index_vars)):
        for idx2, (s2, index_var2) in enumerate(zip(instance, index_vars)):
            if idx2 >= idx1:
                continue
            table_idx1_idx2 = dpmodel.add_int_table(boundtable_scs2(s1, s2))
            exprs.append(table_idx1_idx2[index_var1, index_var2])

    bound = didppy.IntExpr(0)
    for expr in exprs:
        bound = didppy.max(bound, expr)

    return bound

In [ ]:
def boundexpr_charcount(
    instance: list[str],
    dpmodel: didppy.Model,
    index_vars: list[didppy.ElementVar]
) -> didppy.IntExpr:
    chars = sorted(list(set("".join(instance))))

    # alpha_counts[char][s_i][idx]
    alpha_counts: list[list[list[int]]] = [
        [
            [0] * (len(s) + 1)
            for s in instance
        ]
        for char in chars
    ]

    for cidx, c in enumerate(chars):
        for sidx, s in enumerate(instance):
            alpha_counts[cidx][sidx][len(s)] = 0
            for i in range(len(s) - 1, -1, -1):
                alpha_counts[cidx][sidx][i] = alpha_counts[cidx][sidx][
                    i + 1
                ] + (1 if s[i] == c else 0)

    bound = didppy.IntExpr(0)
    for cidx, _ in enumerate(chars):
        expr = didppy.IntExpr(0)
        for sidx, _ in enumerate(instance):
            table_i = dpmodel.add_int_table(alpha_counts[cidx][sidx])
            expr = didppy.max(expr, table_i[index_vars[sidx]])
        bound += expr

    return bound

In [ ]:
class Model:
    def __init__(
        self, instance: list[str], bound_option: BoundOption = BoundOption.SCS2LEN
    ):
        chars = sorted(list(set("".join(instance))))

        dpmodel = didppy.Model(maximize=False, float_cost=False)

        index_types = [dpmodel.add_object_type(number=len(s) + 1) for s in instance]
        index_vars = [
            dpmodel.add_element_var(object_type=index_type, target=0)
            for index_type in index_types
        ]

        instance_table = dpmodel.add_element_table(
            [
                [
                    chars.index(c) for c in s
                ] + [len(chars)]
                for s in instance
            ]
        )

        dpmodel.add_base_case(
            [
                index_var == len(s)
                for s, index_var in zip(instance, index_vars)
            ]
        )

        # 文字 char に従って進む
        for id_char, char in enumerate(chars):
            condition = didppy.Condition(False)
            for sidx, index_var in enumerate(index_vars):
                condition |= (instance_table[sidx, index_var] == id_char)
            trans = didppy.Transition(
                name=f"{char}",
                cost=1 + didppy.IntExpr.state_cost(),
                effects=[
                    (
                        index_var,
                        (
                            instance_table[sidx, index_var] == id_char
                        ).if_then_else(index_var + 1, index_var),
                    )
                    for sidx, index_var in enumerate(index_vars)
                ],
                preconditions=[condition],
            )
            dpmodel.add_transition(trans)

        # Dual Bound
        if bound_option & BoundOption.MAX_LEN:
            # 残っている文字数が最長のものが下限
            dpmodel.add_dual_bound(boundexpr_maxlen(instance, dpmodel, index_vars))
        if bound_option & BoundOption.SCS2LEN:
            # 残っている文字列から 2 つを選んで SCS を取って長さが最大の物が下限
            # primal solution は 3 つの中で一番良いけど best bound は弱い
            dpmodel.add_dual_bound(boundexpr_scs2len(instance, dpmodel, index_vars))
        if bound_option & BoundOption.CHAR_COUNT:
            # 文字列とアルファベットごとに残数をカウントし, その最大値をアルファベット全体にわたって足したものが下限
            # best bound は良くなるけど primal solution はそんなに良くない
            dpmodel.add_dual_bound(boundexpr_charcount(instance, dpmodel, index_vars))

        self.instance = instance
        self.dpmodel = dpmodel
        self.dpsolver = None
        self.solution = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.dpsolver = didppy.CABS(self.dpmodel, threads=12, time_limit=time_limit, quiet=(not log))
        self.solution = self.dpsolver.search()
        return self

    def to_solution(self) -> str:
        return "".join([trans.name for trans in self.solution.transitions])

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60, log: bool = False) -> str:
    model = Model(instance)
    model.solve(time_limit, log)
    return model.to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
model_01 = Model(instance_01)
solution_01 = model_01.solve().to_solution()

In [ ]:
_instance = instance_01
_model = model_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: ultkcignkyuhcosjoemviqfaozgpplnbrxdxdbcsvrvnnshtuqgpxzvxissbxf
str1: --tk--gnk-uh------m--------p-----x---------n--ht-qg-xzvxis----
str2: -----i-------o-j----iqf-o----lnb-x-x--c-v----s--uq-p--v-issbxf
str3: ul--ci-n-y--cos-o--v----oz-ppl---------------------p----------
str4: -----ig----------e-v---a-zg----br-d-dbcsvrvnn-----g----------f

solution is feasible: True
solution is optimal: True
best bound: 62


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
model_02 = Model(instance_02)
solution_02 = model_02.solve().to_solution()

In [ ]:
_instance = instance_02
_model = model_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 99) ---
 Sol: iojiqfopyplrtkgbdenbxvazwxqkrdurlcphmqvzfgjtdbpfuivxernzdsychodutmsbopcsvroqgxzpvnxnigplesswbpbxhdf
str1: ------------tkg---n--------k--u----hm---------p----x--n-----h---t----------qgxz-v-x-i----s---------
str2: iojiqfo---l-------nbx----x-------c----v------------------s-----u-----------q---pv---i----ss-b--x--f
str3: ------------------------------u-lc---------------i----n---yc-o----s-o---v-o---zp------pl-----p-----
str4: i-------------g--e---vaz-----------------g---b-------r--d-----d----b--csvr------vn-n-g------------f
str5: -------pyplr-----------z-x----u--cp-mqv--g-td--fuiv--------c--d---sbo------------------------------
str6: -------p-------bde---v-------d---c----v-----d-pf-------z-s--

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
model_03 = Model(instance_03)
solution_03 = model_03.solve().to_solution()

In [ ]:
_instance = instance_03
_model = model_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 149) ---
  Sol: pyplrsxutiqobdzkjwaisgexqfolnbvckurdafzcpgqafjhtvdbmpxierfngxlequycdvjzseghtiwaoduqmbkvncestfzompbuvhdgikskrjvozqxizucpwvdsbngphxbaknghzlwidcompyfsge
str01: --------t------k-----g------n---ku------------h----mpx----n---------------ht------q-------------------g----------x-z----v-------x---------i-------s--
str02: ---------i-o----j--i----qfolnb-----------------------x------x-----c-v--s---------uq-------------p--v---i-s----------------sb----x----------------f---
str03: ---

In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
model_04 = Model(instance_04)
solution_04 = model_04.solve().to_solution()

In [ ]:
_instance = instance_04
_model = model_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 27) ---
  Sol: bbaedcdbeacdeecbdbeacbdcade
str01: ----dc-b--c---c-db--c--c--e
str02: b---d-dbe---ee----e--bd----
str03: -----c---acdeec---e--b----e
str04: --aed-d----d----d-e--bd--d-
str05: --a--c-be---e-c----a-b-c--e
str06: bba----be------bd---cb--a--
str07: bbae-----a--e--b---a--d-a--
str08: ---e----e---eecbdbe-------e
str09: -----c----cdee--d--a--dc-d-
str10: b---d----a-----bdbea----ad-

solution is feasible: True
solution is optimal: True
best bound: 27


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
model_05 = Model(instance_05)
solution_05 = model_05.solve().to_solution()

In [ ]:
_instance = instance_05
_model = model_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 34) ---
  Sol: dbaecdeabecdabceadecbdeabea

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
model_06 = Model(instance_06)
solution_06 = model_06.solve().to_solution()

In [ ]:
_instance = instance_06
_model = model_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TACTACGCGTAGATCAGACTGTAC
str01: -A-T--G-G--GAT-A--C-G---
str02: -A-TAC-C-T---TC---C----C
str03: --C-ACG---A-AT-----TG-A-
str04: TA--A-----A-ATC----TGT--
str05: -A----G-GTA-A-CA-A----A-
str06: T--T-C-C-TAG----G--T--A-
str07: T--T--G--TAGATC----T----
str08: T-----G-G--GA--AG--T-T-C
str09: T--T-C-C--A---CA-ACT----
str10: T-CTA-----A-A-C-GA----A-

solution is feasible: True
solution is optimal: True
best bound: 24


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
model_07 = Model(instance_07)
solution_07 = model_07.solve().to_solution()

In [ ]:
_instance = instance_07
_model = model_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

-A--CATC----TCA---A-TA-C--A--A--C-AT-A--A--G--AA----AA--AC----A-A-C--GCA----A---A----A--AC----AC-T---C---AT---------
str23: A---A---A----CG-A-A--C-T-----T---T-A--A----A--AT-C--T--GT---G-T----G---G--C-TG--T--CA--C--TCG--GC--TGCA-T----G--C-T----T--A---GT-G-C-
str24: A-T-A---A----C-TA-AT---T-A--CT--GTC-GT---TGA-CA-G-GA-CA---C-G--A---G----TA-----A---C-T-C-GTC--T--A-T-C--T--TC-T--G-------------------
str25: A-TGA-GT-GT--C--AC--G-A--A---T---TCA----C-G-T-A--C-A--A-T---G--AACTG---G-A--TG--T----T-CA--CG-TG----G-A--A-T---A---A-----------------
str26: AC---CGT-G----G-----GC----GA----G-C-G--G-TGA-C---CG----GT---G-T--CT-----T-CCT--A--G--TG--G--G-T-C-C--CA---C--GT---T-GA----A---------R
str27: A---A---AG----GT---T---T-A---T-A--C-----CT--TC---C---CAG----G-TAAC---AA--ACC---A-A-C---CA----A--C--T----T--TCG-A--T---CT-C-T---T-G---
str28: A--G---TAGT----T-C--GC--C----T--GT--GT-G---A----GC--T--G-ACA---AACT-----TA---G--TAG--TG---T---T----TG---T----G-A-G--GA-T---T----A----
str29: --T----T--TA---TAC---C-T-----T

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
model_08 = Model(instance_08)
solution_08 = model_08.solve().to_solution()

In [ ]:
_instance = instance_08
_model = model_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 44) ---
  Sol: MQPSKFTRAESLNESYAQHFVDANISRCPGVKFGTNEALHDGYQ
str01: M-------A--L--SY-----------CP--K-GT---------
str02: MQ-S------SLN---A-------I---P-V-------------
str03: M-P--------L--SY-QHF------R----K------------
str04: M--------E---E----H-V--N------------E-LHD---
str05: M--S--------N------F-DA-I-R----------AL-----
str06: M----F-R----N----Q-----N-SR--------N-----G--
str07: M----F---------YA-H---A---------FG-------GY-
str08: M--SKFTR------------------R-P-------------YQ
str09: M--S-F--------------V-A------GV---T--A-----Q
str10: M--------ESL--------V-------PG--F--NE-------

solution is feasible: True
solution is optimal: False
best bound: 29


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
model_09 = Model(instance_09)
solution_09 = model_09.solve().to_solution()

In [ ]:
_instance = instance_09
_model = model_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

-----------G-E----T------------R-I-----------------I----------------R----------I--------------------------T---G------S---------------------------------------
str07: M------A---------FD----------------------------F----------S--V----------------T--G----N-----------T----------------------K-LD---------T--------------------S------G-----------F-------T-----------------------Q--------G---------V--------------S-----S-------M---------T---------------------V---A------A---G---T----L--------I-A------------D-------LV----------------KT-----------A-----------------S----------------S--------------------------------Q----L--T---------N----L-A-------------Q--S----------------
str08: M------A--------V-------------I------L--------------P-----S-------------------T----------------Y--TD------------G---------------------T----------AA-----------C-----------------------TNG-----------------S--P------D------------V-----------------------V---------G----TG---------T---------M-------------W-----------------V----NT---

----------RG------A-----------DA------------------------------P-----A-----F-------------------------------------Q---D--------T-----------A---------------N-------------Q-------QA--R----------Q-------------------
str17: M--F------------VF-----------L---------------------V-L-----------L---------------------------P---L-----V--------------S--------S----Q---------C--------------V-------------------------N------L-----R-T-----R----T-----------Q-----L-P----L------A--------------------------Y------T----------------N--SF--------T-----R--------------------GV------------Y---------------Y---------P-------D--K---------V---------F-------R------------S-------------------S----------V--L--H---------S----------------------------
str18: M--F------------VF-----------------------------F---V-L-----------L---------------------------P---L-----V--------------S--------S----Q---------C--------------V-------------------------N------L-------T----------T------------R---T------QL--------------------P--------------PA--

--S---------------------------Y------T----------------N--SF--------T-----R--------------------GV------------Y---------------Y---------P-------D--K---------V---------F-------R------------S-------------------S----------V--L--H---------S----------T---Q----D--------
str27: M-------K--------FD--------------------------------V-L----S------L-----------------------F--AP-----------------W-------A-KV-D----------------E------------Q-------------E------Y-D---Q------------------------QL-----------------------N-------------N-----N------------------------------LE-----------S----------I----------------T-A--------------------------P-------K---F--D------------D--------------G---------A---------T-----EI--E------------------S---------E--------------R---------G-----------D-I------
str28: M--F------------VF-----------L---------------------V-L-----------L---------------------------P---L-----V--------------S--------S----Q---------C--------------V-------------------------N----F---------T--N--R----T-----------Q

-----PF--Q------S---------L--K-P------V----------------------M----A-N----A------------L---------------------GV--------L--------E--G-----K-----M------------------F-------------C--------S-------------I----------G-------------------G---------------R-S-----------------L----------
str33: M------A--------------------------T--L---------------L--R-S------L--A--------------L-----F----K-R------------N-----------K--D------------------KP-------------------------P--I--------T-------------------S------------G-S-G-----------------------G-------------A---I-----R-------------------------G------------I-------K------------H------------I------------I-I---------V------P---I--------------------P--G-----D-S---------------S----------------------I-T------------T------R-S-------------R--------------
str34: ME------------S--------------L---------------------VP-----------------G------------------F-------------------N------E----K------------T-HV----------------Q-----L-------------------S---------L--P---V---------L

----------V---------------------------V------------------------------E------K----------------K----------T-G----D---------A----------I-----------S--D---------D------E------------------N----------E-----N--------D-----------S----------D-----------------------------T---GE-D-L----------------V--D---------------------
str39: M--F------------VF-----------L---------------------V-L-----------L---------------------------P---L-----V--------------S--------S----Q---------C--------------V-------------------------N------L-----R-T-----R----T-----------Q-----L-P-------------------------P-------------------------S------Y--TN--SF--------T-----R--------------------GV------------Y---------------Y---------P-------D--K---------V---------F-------R------------S-------------------S----------V--L--H---------S----------------------------
str40: ME------------S--------------L---------------------VP-----------------G------------------F-------------------N------E----K------------T-HV----------------Q-----L----------

----Q---------------P----------------------P-------------L-----A---S-G--------------------S--GG------------A---I-----R-------------------------G------------I-------K------------H-----V------I------------I-----------V---L------I--------------------P--G-----D-S---------------S----------------------I-------V------T------R-S-------------R--------------
str47: M-----------R---V----R----G---I------LR------N---------------------W---------Q-------------Q-------------------W------------------W----I---W-----------T---S----L-G-----------F-----------------------------------W---M---------------F-----------------------M------I----C--------------S----VV-----G----------------------------N------LW--V---------------------------T---V---------Y----------------Y--G--------------V-----P------V---------------W-----K--------E-----------A-------K-------T---------T-------
str48: M------A--------V-----EP-----------------------F----P---R-------------------------R----------P-------I--------T--------------------R--